# Downloading data and unzipping

In [3]:
# !gdown https://drive.google.com/uc?id=1STTlixkDCgeUMgWqrrYmiwaa3w6B9MFe&export=download

In [4]:
# !unzip assignment_1_data.zip

# Importing input data

In [5]:
import json
import re

In [6]:
with open('assignment_1_data/input.json','r', encoding='utf8') as input_file:
  input_data = json.load(input_file)
  input_file.close()

In [7]:
with open('assignment_1_data/output.json','r', encoding='utf8') as output_file:
  output_data = json.load(output_file)
  output_file.close()

In [8]:
print(input_data[7])
print(output_data[0])

{'sid': 7, 'input_tokens': ['The', 'population', 'density', 'was', '18', 'people', 'per', 'square', 'mile', '(', '7/km²', ')', '.']}
{'sid': 0, 'output_tokens': ['<self>', '<self>', 'one hundred', '<self>', '<self>', '<self>', 'ninety point zero', '<self>', 'sil']}


# Sample functions for generating the output

- If there is a numeric inside the string, the check for the following:
    - If there is no other character except for numerics, then it should be handled like a number.
    - If there is a symbol in front of numerics like symbols of currency, then they should be handled as per currency.
    - If there are any characters, then the string can either be a date or might be having some units associated with the numeric, eg: 5m.

In [9]:
# # Return the text version associated to the numeric in the input string
# def convert_number(input_string):
    

In [10]:
# roman_numerals = {
#     'I': 'first',
#     'II': 'second',
#     'III': 'third',
#     'IV': 'fourth',
#     'V': 'fifth',
#     'VI': 'sixth',
#     'VII': 'seventh',
#     'VIII': 'eighth',
#     'IX': 'ninth',
#     'X': 'tenth',
#     'XI': 'eleventh',
#     'XII': 'twelfth',
# }

In [11]:
roman_numerals = {
    'I': '1',
    'II': '2',
    'III': '3',
    'IV': '4',
    'V': '5',
    'VI': '6',
    'VII': '7',
    'VIII': '8',
    'IX': '9',
    'X': '10',
    'XI': '11',
    'XII': '12',
}

In [12]:
full_form = {
        'm': 'meter', 
        'mm': 'millimeter',
        'cm': 'centimeter',
        'km': 'kilometer',
        'mi': 'mile',

        'kmph': 'kilometers per hour',
        'mph': 'miles per hour',

        'g': 'gram',
        'kg': 'kilogram',
        'mg': 'milligram',

        'm2': 'square meters',
        'mm2': 'square millimeters',
        'cm2': 'square centimeters',
        'km2': 'square kilometers',
        'mi2': 'square miles',
        'km²': 'square kilometers',
        
        'sq': 'squares',
        'sqm': 'square meter',
        'sqcm': 'square centimeter',
        'sqkm': 'square kilometer',
        'sqmi': 'square mile',
        'ha': 'hectare',
        'ac': 'acre',

        'm3': 'cubic meter',
        'mm3': 'cubic millimeter',
        'cm3': 'cubic centimeter',
        'km3': 'cubic kilometer',

        's': 'second',
        'min': 'minute',
        'rpm': 'revolutions per minute',

        'a.m.': 'a m',
        'p.m.': 'p m',
        'am': 'a m',
        'pm': 'p m',
        'AM': 'a m',
        'PM': 'p m',

        'M': 'million',
        'mn': 'million',
        'Mn': 'million',

        'B': 'billion',
        'b': 'billion',
        'bn': 'billion',
        'Bn': 'billion',
        'cr': 'crore',

        'KB': 'kilobyte',
        'MB': 'megabyte',
        'GB': 'gigabyte',
        'TB': 'terabyte',
        'PB': 'peta byte',              # sid 158

        'Kb': 'kilobit',
        'Mb': 'megabit',
        'Gb': 'gigabit',
        'Tb': 'terabit',
        'TB': 'petabit',

        'Kbps': 'kilobits per second',
        'Mbps': 'megabits per second',
        'Gbps': 'gigabits per second',
        'Tbps': 'terabits per second',

        'mA': 'milliampere',

        'N': 'North',
        'E': 'East',
        'W': 'West',
        'S': 'South',
        'NE': 'North East',
        'NW': 'North West',
        'SW': 'South East',
        'SE': 'South West'
        }

In [13]:
numbers = {
    '-': 'minus',
    '0': 'zero',
    '1': 'one',
    '2': 'two',
    '3': 'three',
    '4': 'four',
    '5': 'five',
    '6': 'six',
    '7': 'seven',
    '8': 'eight',
    '9': 'nine',
    '10': 'ten',
    '11': 'eleven',
    '12': 'twelve',
    '13': 'thirteen',
    '14': 'fourteen',
    '15': 'fifteen',
    '16': 'sixteen',
    '17': 'seventeen',
    '18': 'eighteen',
    '19': 'nineteen',
    '20': 'twenty',
    '30': 'thirty',
    '40': 'forty',
    '50': 'fifty',
    '60': 'sixty',
    '70': 'seventy',
    '80': 'eighty',
    '90': 'ninety'
}

In [14]:
extended_nums = {
    '0': 'ieth',         # y is replaced by this if the last numeric is 0
    '1': 'first',
    '2': 'second',
    '3': 'third',
    '4': 'fourth',
    '5': 'fifth',
    '6': 'sixth',
    '7': 'seventh',
    '8': 'eighth',
    '9': 'ninth'
}

In [15]:
def is_symbol(input_string):
    if bool(re.search(r'\$', input_string)):
        return True, 'dollar'
    elif bool(re.search(r'\£', input_string)):
        return True, 'pound'
    elif bool(re.search(r'\€', input_string)):
        return True, 'euro'
    elif bool(re.search(r'(Rs[ \.]){1}', input_string)):
        return True, 'rupee'
    else:
        return False, False

In [16]:
is_symbol('Rs 50,000')

(True, 'rupee')

In [17]:
is_symbol('€349')

(True, 'euro')

- If there are two 

In [18]:
# Returns if a punctuation is present in the input string
def has_punc(input_string):
    return bool(re.search(r'[^\w\s]', input_string))

In [19]:
def get_punc(input_string):
    return 'sil'

In [20]:
# Returns true if the input string has a numeric present
def has_numeric(input_string):
    return bool(re.search(r'\d', input_string))

In [21]:
# Returns true if the input string has only 4 numerical characters, which would be indicative of an year
def has_fournumeric(input_string):
    return bool(re.search(r'^\d{4}$', input_string))

In [22]:
def get_fournumeric(input_string):
    if len(input_string) == 4:
        if input_string[1:3] == '00':               # handles 2007 --> 'two thousand seven'
            return get_onlydigits(input_string)
        else:                                       # handles 1906 --> 'nineteen o six'; but 2012 --> 'twenty twelve'
            first_two = input_string[0:2]
            last_two = input_string[-2:]
            if get_onlydigits(first_two) != 'zero':
                if input_string[2] == '0':              # handles 1906 --> 'nineteen o six';
                    return get_onlydigits(first_two) + ' o ' + get_onlydigits(last_two)
                else:                                   # handles 2012 --> 'twenty twelve';
                    return get_onlydigits(first_two) + ' ' + get_onlydigits(last_two)
            else:                                       # handles 0002 --> 'two';
                return get_onlydigits(last_two)
    else:
        return ''

In [23]:
get_fournumeric('006')

''

In [24]:
# Return true if the input string has only numerical digits
def has_onlydigits(input_string):
    # sid 110 can tell that the first digit has to be greater than 0
    return bool(re.search(r'^(-|[1-9])\d*[,\d]*\d*$', input_string))            

In [25]:
has_onlydigits('05')

False

In [26]:
# Return true if the input string has a sequence similar to a ISBN codes that have a dashed sequence.
def has_isbncode(input_string):
    # If the first digit is zero, then the number can probably be for isbn; sid 110
    return bool(re.search(r'^\d+[ |-]\d+', input_string))        # Since we want to find a repeating pattern, either the start or the end fixer has to be relaxed

In [27]:
# The case for dates in the same format still need to be handled
has_isbncode('0252076729')

False

In [28]:
def get_isbncode(input_string):
    res = ''
    for char in input_string:
        # Punctuation case needs to be handled first, else '-' would be taken as 'minus' from the numbers dictionary
        if has_punc(char) or char == ' ':
            res += ' ' + get_punc(char)
            continue
        
        # '0' case needs to be handled before other numerals since this is converted to 'o' rather than 'zero'
        if char == '0':
            res += ' o'
        
        else:
            res += ' ' + numbers[char]
    
    res = ' '.join(res.split())             # to remove extra spaces at the front or end
    return res

In [29]:
get_isbncode('803.05')

'eight o three sil o five'

In [30]:
def get_onlydigits(input_string):
    input_string = str(input_string)
    
    # To ensure that no empty strings give any issue
    if input_string.strip() == '':
        return ''

    res = ''
    input_string = re.sub(r'[^\d-]', '', input_string)
    int_string = int(input_string)
    
    if int_string < 0:
        res += 'minus '                     # there would surely be something in front of the negative sign here.
        input_string = input_string[1:]
        int_string *= -1

    # The case with only 4 digits needs to be incorporated; 1991 --> nineteen ninety one; the first two numerals are separated from the other two

    # Handling numbers directly from dictionary
    if int_string <= 20:
        res += numbers[str(int_string)]     # for example: '07' means '7' and thus has to be looked this way in the dictionary
        return res                          # since we do not need to do anything further from here


    # Handling two digit numbers not in dictionary
    if int_string < 100:
        if str(int_string) in numbers.keys():       # '30' --> 'thirty'
            res += numbers[str(int_string)]
            return res
        else:
            tenth_place = (int_string//10) * 10     # '35' --> 'thirty five'
            unit_place = (int_string % 10)
            res += numbers[str(tenth_place)] + ' ' + numbers[str(unit_place)]
            return res

    t = 1000
    # Handling hundreds
    if int_string < t:
        hundred_place = int_string // 100
        if int_string % 100 == 0:                   # '900' --> 'nine hundred'
            res += numbers[str(hundred_place)] + ' hundred'
            return res
        else:
            res += numbers[str(hundred_place)] + ' hundred ' + get_onlydigits(str(int_string % 100))
            return res

    # Handling thousands
    # As the number increases, the number of zeroes becomes over-whelming! Better to convert it into a variable
    m = t*t
    if int_string < m:
        thousand_place = int_string // t
        if int_string % t == 0:                   
            res += get_onlydigits(str(thousand_place)) + ' thousand' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(thousand_place)) + ' thousand ' + get_onlydigits(str(int_string % t))
            return res

    # Handling millions
    b = m*t
    if int_string < b:
        million_place = int_string // m
        if int_string % m == 0:                   
            res += get_onlydigits(str(million_place)) + ' million' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(million_place)) + ' million ' + get_onlydigits(str(int_string % m))
            return res

    # Handling billions
    tr = b*t
    if int_string < tr:
        billion_place = int_string // b
        if int_string % b == 0:                   
            res += get_onlydigits(str(billion_place)) + ' billion' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(billion_place)) + ' billion ' + get_onlydigits(str(int_string % b))
            return res

    # Handling trillions
    pr = tr*t
    if int_string < pr:
        prillion_place = int_string // tr
        if int_string % tr == 0:                   
            res += get_onlydigits(str(prillion_place)) + ' trillion' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(prillion_place)) + ' trillion ' + get_onlydigits(str(int_string % tr))
            return res

    # Check if numbers greater than billions appear in the corpus often; ref: GDP is also only in trillions!

    # Symbols need to be incorporated; $20 --> '20 dollars'; $1 --> 'one dollar'


    # Check if the numbers can also come with thousand separators like 1,000,000

    # 'res +=' can also be removed since we are not making any use of that operation!

    return res

In [31]:
' '.strip()

''

In [32]:
get_onlydigits(21)

'twenty one'

In [33]:
get_onlydigits(19)

'nineteen'

In [34]:
def has_decimal(input_string):
    return bool(re.search(r'^\d+[,]{0,1}\d*[.]\d+$', input_string))      # ending on a digit is justified since period (.) is a different token in the corpus

In [35]:
has_decimal('3,221.221')

True

In [36]:
a = '0.153'
a.split('.')

['0', '153']

In [37]:
def get_decimal(input_string):
    res = ''
    num_list = input_string.split('.')
    # For ex: '90.0' --> 'ninety point zero' and not 'ninety point o'
    if bool(re.search(r'^0+$', num_list[1])):
        res = res + get_onlydigits(num_list[0]) + ' point ' + 'zero '*len(num_list[1])
    else:
        res = res + get_onlydigits(num_list[0]) + ' point ' + get_isbncode(num_list[1])
    return ' '.join(res.split())

In [38]:
get_decimal('31,212.00')

'thirty one thousand two hundred twelve point zero zero'

In [39]:
get_decimal('31212.01')

'thirty one thousand two hundred twelve point o one'

In [40]:
def has_units(input_string):
    return bool(re.search(r'^\d+.*[a-zA-Z]+', input_string))

In [41]:
re.split(r'\s+|[ /]\s*', '100 KB/s')

['100', 'KB', 's']

In [42]:
'100Kb/s'.split(' ')

['100Kb/s']

In [43]:
re.findall("([0-9]+|[A-Z]+)", "5GB")

['5', 'GB']

In [44]:
def get_units(input_string):
    res = ''
    is_per = False
    plural = False
    
    # '50.0Kb Gb/s' --> ['50.0Kb', 'Gb/s']
    # input_string = re.sub(r'\.', '', input_string)
    words_list = input_string.split(' ')
    

    # if bool(re.search(r'/', input_string)):
    #     is_per = True

    # if re.search(r'/[a-zA-Z]', input_string):       # for ex: '70.05/sqkm' --> 'seventy point zero five per square kilometer'
    #     words_list = re.split(r'\s+|[ /]\s*', input_string)
    #     is_per = True
    
    # if not is_per:
    #     words_list = input_string.split(' ')
    #     is_space = True

    for word in words_list:
        # print(word)
        # Separating the number; for ex: '50.0Kb' --> ['50.0', 'Kb']
        key_list = re.findall('([0-9.,/]+|[a-zA-Z].*)', word)
        for key in key_list:
            # print(key)
            if bool(re.search(r'/', key)):
                temp_list = key.split('/')
                left = True
                for temp in temp_list:
                    # print(temp)
                    if not left:
                        res += 'per '
                    # if temp == '':
                    #     continue
                    if has_decimal(temp):
                        res += get_decimal(temp) + ' '
                    elif has_onlydigits(temp):
                        res += get_onlydigits(temp) + ' '
                    elif temp in full_form.keys():
                        if plural and left and ((temp != 'a.m.') and (temp != 'p.m.')):
                            res += full_form[temp] + 's '        # won't affect am, pm, etc.
                        else:
                            res += full_form[temp] + ' '
                    else:
                        res += temp + ' '
                    left = False
            
            elif has_decimal(key):
                # print(key)
                # Since it can also be '8 a.m.' where 'a.m.' --> 'a m'
                if key in full_form.keys():
                    res += full_form[key] + ' '
                # Otherwise, we are talking about floating point number, for ex: 80.3
                else:
                    res += get_decimal(key) + ' '
                    if (int(key[0]) > 1) or (len(key) > 1):
                        plural = True                       # 5 kilometers
            
            elif has_onlydigits(key):
                res += get_onlydigits(key) + ' '  
                if (int(key[0]) > 1) or (len(key) > 1):
                    plural = True                       # 5 kilometers
                             
            elif key in full_form.keys():
                if plural and ((re.sub('\.', '', key.lower()) != 'am') and (re.sub('\.', '', key.lower()) != 'pm')):
                    res += full_form[key] + 's '        # won't affect am, pm, etc.
                else:
                    res += full_form[key] + ' '
            
            else:
                res += key + ' '

    return ' '.join(res.split())

In [45]:
get_units('9:30 AM')

'nine thirty a m'

In [46]:
get_units('604.2/km2')

'six hundred four point two per square kilometers'

In [47]:
re.findall('([0-9.,/]+|[a-zA-Z].*)', 'm')

['m']

In [48]:
get_units('100Gb/s')

'one hundred gigabits per second'

In [49]:
get_units('179.4/km2')

'one hundred seventy nine point four per square kilometers'

In [50]:
get_units('823.08 KB')

'eight hundred twenty three point o eight kilobytes'

In [51]:
def has_extendednums(input_string):
    return bool(re.search(r'\d+[ ]*(st|nd|rd|th)', input_string))

In [52]:
'00209'.lstrip('0')

'209'

In [53]:
# Handles the cases of extended numerals; for ex: '100th' --> 'one hundredth'
def get_extendednums(input_string):
    # Getting the last numeric digit present in the string
    numeric = re.sub(r'[^\d]', '', input_string)
    numeric = numeric.lstrip('0')         # to remove initial zeroes in the string
    numeric_string = str(numeric)
    non_unit_numeric = (int(numeric) // 10) * 10

    if numeric_string == '12':
        return 'twelfth'

    # For ex: 02nd --> 2, then this only has one digit
    if numeric_string in extended_nums.keys():
        if numeric_string == '0':
            return 'zeroth'
        return extended_nums[numeric_string]
    
    unit_numeric = numeric[-1]

    # If the last two digits are between 10 and 20 (both inclusive), then we have to get its direct form
    last_two_digits = int(numeric[-2:])
    if len(numeric) == 2 and (last_two_digits >= 10) and (int(last_two_digits) < 20):
        return get_onlydigits(str(last_two_digits)) + 'th'
    elif len(numeric) > 2 and (last_two_digits >= 10) and (int(last_two_digits) < 20):
        return get_onlydigits((int(numeric) // 100) * 100) + ' ' + get_extendednums(str(last_two_digits))
    else:
        # Getting the translation for the non-last digit
        non_unit_string = get_onlydigits(str(non_unit_numeric))
        # Handling case with last two digits as 0; for ex: 100th --> 'one hundreth' and not 'one hundreith'
        if last_two_digits == 0:
            return get_onlydigits((int(numeric) // 100) * 100) + 'th'
        # Handling case with last two digits as 0; for ex: 130th --> 'one hundred thirieth' and not 'one hundred thirtyth'
        elif unit_numeric == '0':
            return non_unit_string[:-1] + extended_nums[unit_numeric]
        else:
            return non_unit_string + ' ' + extended_nums[unit_numeric]

In [54]:
get_extendednums('02')

'second'

In [55]:
def has_allcaps(input_string):
    return ((len(input_string) > 2) and bool(re.search(r'^[A-Z]{2,}[a-z]{1,2}\.*$', input_string))) or (bool(re.search(r'^[A-Z]+\.*$', input_string))) or (bool(re.search(r'[A-Z][\.-]$', input_string)))

In [56]:
has_allcaps('NPAeep')

False

In [57]:
# Converst the abbreviated version to a spaced version as per output format
def get_allcaps(input_string):                          # ref: sid 202
    input_string = re.sub(r'[\.-]', '', input_string)
    res = ((' '.join(input_string)).lower())
    return ' '.join(res.split())

In [58]:
get_allcaps('S. A. ')

's a'

In [59]:
' '.join('s   a  '.split())

's a'

In [60]:
# Returns true if the input string forms a part of an abbreviation
def get_abbr(input_string):
    if input_string in full_form.keys():
        return full_form[input_string]

In [61]:
# bool(re.search(r'[A-Z]\.', input_string)

In [62]:
get_abbr('sq')

'squares'

In [63]:
has_numeric('He has #22# dogs')

True

In [64]:
def has_time(input_string):
    return bool(re.search(r'^\d{1,2}: *\d{1,2}', input_string))

In [65]:
has_time('5:43')

True

In [66]:
'5:30 pm PST'.split(' ')

['5:30', 'pm', 'PST']

In [67]:
re.findall('([0-9.,/]+|[a-zA-Z].+)', 'PST')

['PST']

In [68]:
def get_time(input_string):
    res = ''
    hour_24 = False
    input_string = re.sub(r'\.', '', input_string)
    time_list = input_string.split(' ')

    for time in time_list:
        key_list = re.findall('([0-9.,:/]+|[a-zA-Z].+)', time)
        for key in key_list:
            # Handles '5:30' --> 'five thirty'
            if bool(re.search(r':', key)):
                temp_list = key.split(':')
                # Handling sid 201 that for 2:27:07 --> 'two hours twenty seven minutes and seven seconds
                if len(temp_list) == 3:
                    print("hello")
                    counter = 1
                    for temp in temp_list:
                        if has_onlydigits(temp):
                            temp = temp.lstrip('0')             # to remove initial zeroes in the string
                            temp = temp.lstrip(' ')             # to remove initial zeroes in the string
                            if counter == 1:
                                res += get_onlydigits(temp) + ' hours '
                            elif counter == 2:
                                res += get_onlydigits(temp) + ' minutes '
                            elif counter == 3:
                                res += ' and ' + get_onlydigits(temp) + ' seconds'
                            counter += 1                        # indicating that the next time term comes
                else:
                    for temp in temp_list:
                        # 24 hour clock means that 19:00 -> 'nineteen hundrd'
                        if len(temp) >= 1 and int(temp) > 12:   
                            hour_24 = True
                        
                        # sid 118: 19:00 --> 'nineteen hundred'
                        if temp == '00':
                            if hour_24:
                                res += 'hundred'
                            else:
                                res += ''
                        
                        # to remove initial zeroes in the string; except for the case when it is '00'
                        elif has_onlydigits(temp.lstrip('0')):
                            res += get_onlydigits(temp.lstrip('0')) + ' '
            
            # ' 30' --> '30' --> 'thirty'
            elif has_onlydigits(key.lstrip()):
                res += get_onlydigits(key.lstrip()) + ' '

            elif bool(re.search(r'^[a-zA-Z]+$', key)):
                res += get_allcaps(key) + ' '
            else:
                res += key + ' '
    return ' '.join(res.split())

In [69]:
re.findall('([0-9.,:/]+|[a-zA-Z].+)', '12: 30: 50')

['12:', '30:', '50']

In [70]:
len('12: 30: 50'.split(':'))

3

In [71]:
get_time('05:30')

'five thirty'

In [72]:
def has_date(input_string):
    month_day = bool(re.search(r'Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|September|Oct|October|Nov|November|Dec|December', input_string))
    ddmmyyyy = bool(re.search(r'^\d{1,2}[ ,-]+([0]\d{1}|1[012])[ ,-]\d{4}$', input_string))     # no control on daters
    mmddyyyy = bool(re.search(r'^([0]\d{1}|1[012])[ ,-]\d{1,2}[ ,-]+\d{4}$', input_string))
    yyyymmdd = bool(re.search(r'^\d{4}[ ,-]+([0]\d{1}|1[012])[ ,-]\d{1,2}$', input_string))
    
    # Starting with year
    if yyyymmdd:
        year = True
    else:
        year = False

    # Starting with month
    if mmddyyyy:
        month = True
    else:
        month = False
    
    # Starting with day
    if ddmmyyyy:
        day = True
    else:
        day = False
    
    # Has a day component
    if yyyymmdd or mmddyyyy or ddmmyyyy or month_day:
        return True, [day, month, year, month_day]
    else:
        return False, [False, False, False, False]

In [73]:
has_date('2008-08-17')

(True, [False, False, True, False])

In [74]:
bool(re.search(r'^\D+$','January eiufha askefh auet;sadfl iasf/2'))

False

In [75]:
bool(re.search(r'^\d{2}', '20 Jan'))

True

In [76]:
re.split(r'\s+|[,.-]\s*', '20 May.2')

['20', 'May', '2']

In [77]:
# Doesn't handle '1 1/2' --> 'one and a half'
def has_division(input_string):
    return bool(re.search(r'^-*(\d+)\s*/\d+', input_string))

In [78]:
has_division('-79/316')

True

In [79]:
def get_division(input_string):
    res = ''
    numerator = True
    number_list = input_string.split('/')

    for number in number_list:
        if numerator:
            if has_onlydigits(number):
                res += get_onlydigits(number) + ' '
            elif has_decimal(number):
                res += get_decimal(number) + ' '
            numerator = False
        else:
            res += get_extendednums(number) + 's'

    return res      

In [80]:
get_division('17/96')

'seventeen ninety sixths'

In [81]:
month = {
    'jan': 'january',
    'feb': 'february',
    'mar': 'march',
    'apr': 'april',
    'may': 'may',
    'jun': 'june',
    'jul': 'july',
    'aug': 'august',
    'sept': 'september',
    'oct': 'october',
    'nov': 'november',
    'dec': 'december',
}

In [82]:
month_num = {
    '01': 'january',
    '02': 'february',
    '03': 'march',
    '04': 'april',
    '05': 'may',
    '06': 'june',
    '07': 'july',
    '08': 'august',
    '09': 'september',
    '10': 'october',
    '11': 'november',
    '12': 'december',
}

In [83]:
' '.join(' 20 Jan '.split())

'20 Jan'

In [84]:
def get_date(input_string, order):
    # order is of the form [day, month, year, month_day]

    def has_month(input_string):
        return bool(re.search(r'^\D+$', input_string))

    def get_month(input_string):
        if input_string.lower() in month.keys():
            return month[input_string.lower()]
        else:
            return input_string.lower()

    res = ''
    keyword_list = re.split(r'\s+|[,.-]\s*', input_string)
    # Back-filling day, month or year number with 0s; for ex: October 1, 2021 --> October 01, 2021
    for i in range(len(keyword_list)):
        if len(keyword_list[i]) == 1:
            keyword_list[i] = '0' + keyword_list[i]
        elif has_onlydigits(keyword_list[i][:-2]) and (keyword_list[i][-2:] == 'nd' or keyword_list[i][-2:] == 'th' or keyword_list[i][-2:] == 'st' or keyword_list[i][-2:] == 'rd'):
            keyword_list[i] = keyword_list[i][:-2]
            if len(keyword_list[i]) == 1:
                keyword_list[i] = '0' + keyword_list[i]

    # for key in keyword_list:
    #     print(type(key))

    # January 14, 2008 --> january fourteenth two thousand eight
    # 20 January 2010 --> the twentieth of january twenty ten
    # January 2010 --> january twenty ten

    if order[3]:
        # Handles if input string only has the month name like 'January'
        if bool(re.search(r'^\D+$',input_string)):
            return '<self>'                 # since it has no numeral to tell us that we are talking about a specific date
        else:
            # If first keyword is a month name
            if bool(re.search(r'^\D+$', keyword_list[0])):
                if has_month(keyword_list[0]):
                    res += get_month(keyword_list[0])
                
                # If the next keyword is a date, then push its extended form                  
                if bool(re.search(r'^\d{2}$', keyword_list[1])):
                    res += ' ' + get_extendednums(keyword_list[1])
                    date_inlcuded = True

                    # If there exists an year to process
                    if len(keyword_list) > 2:
                        res += ' ' + get_fournumeric(keyword_list[2])
                    
                    return ' '.join(res.split())

                # If the next keyword is a year
                elif bool(re.search(r'^\d{4}$', keyword_list[1])):
                    res += ' ' + get_fournumeric(keyword_list[1])
                    return ' '.join(res.split())


            # If the keyword is a date, then its extended form needs to be used; 20 --> the twentieth
            elif bool(re.search(r'^\d{2}$', keyword_list[0])):
                print(keyword_list[0])
                res = 'the ' + get_extendednums(keyword_list[0]) + ' of '
                date_included = True

                # if has_month(keyword_list[1]):
                res += get_month(keyword_list[1])
                month_included = True

                # If there exists an year to process
                if len(keyword_list) > 2:
                    res += ' ' + get_fournumeric(keyword_list[2])
                    year_included = True
                
                return ' '.join(res.split())
    
    # If the date is presented in ddmmyyyy format
    elif order[0]:
        res += 'the ' + get_extendednums(keyword_list[0].lstrip('0')) + ' of ' + month_num[keyword_list[1]] + ' ' + get_fournumeric(keyword_list[2])
        return res

    # If the date is presented in mmddyyyy format
    elif order[1]:
        res += 'the ' + get_extendednums(keyword_list[1].lstrip('0')) + ' of ' + month_num[keyword_list[0]] + ' ' + get_fournumeric(keyword_list[2])
        return res

    # If the date is presented in yyyymmdd format
    elif order[2]:
        res += 'the ' + get_extendednums(keyword_list[2].lstrip('0')) + ' of ' + month_num[keyword_list[1]] + ' ' + get_fournumeric(keyword_list[0])
        return res

In [85]:
has_date('2000-08-17')

(True, [False, False, True, False])

In [86]:
def get_symbol(input_string, currency):
    res = ''
    plural = False
    if currency == 'rupee':
        input_string = input_string[2:].lstrip(' ').lstrip('.')
    else:
        input_string = input_string[1:]
    currency_list = input_string.split(' ')

    for curr in currency_list:
        key_list = re.findall('([0-9.,]+|[a-zA-Z].*)', curr)
        for key in key_list:
            if has_decimal(key):
                if float(key) >= 10.0:
                    plural = True
                res += get_decimal(key) + ' '
            elif has_onlydigits(key):
                if len(key) > 1:
                    plural = True
                res += get_onlydigits(key) + ' '
            elif key in full_form.keys():
                if key == 'm':
                    res += ' million '
                else:
                    res += full_form[key] + ' '
                plural = True                       # since it might be 1 million euros
                
            else:
                plural = True
                res += key + ' '
    
    if plural:
        res += currency + 's'
    else:
        res += currency                     # to remove making the currency plural
    return ' '.join(res.split())

In [87]:
get_symbol('€1m', 'euro')

'one million euros'

In [88]:
get_symbol('Rs 1 million', 'rupee')

'one million rupees'

In [89]:
def has_percent(input_string):
    return bool(re.search(r'%', input_string)) or bool(re.search(r'\d(pc)+$', input_string))

In [90]:
has_percent('1-881089-97')

False

In [91]:
def get_percent(input_string):
    res = ''
    input_string = input_string[:-1]            # removing the % or c symbol first
    
    if type(input_string[-1]) == 'str':
        input_string = input_string[:-1]            # removing the p symbol then

    if has_decimal(input_string):
        res += get_decimal(input_string)
    else:
        res += get_onlydigits(input_string)

    return res + ' percent'

In [92]:
get_percent('5.04pc')

'five hundred four percent'

In [93]:
#todo: write your own solution.
def solution(input_tokens):
  sol = []
  flag = False

  for token in input_tokens:

    # All the extra spaces at the start and end of the token should be trimmed since the tokenisation may have a problem (for ex: sid 85 --> "21 ")
    token = ' '.join(token.split())
    
    # Handling Punctuation.
    if((has_punc(token) and (len(token) == 1)) and not flag):
      sol.append(get_punc(token))
      flag = True
    
    # Handling roman numerals.
    if token in roman_numerals.keys() and not flag:
      # sol.append('the ' + roman_numerals[token])
      # sol.append(get_onlydigits(roman_numerals[token]))
      sol.append('the ' + get_extendednums(roman_numerals[token]))
      flag = True

    # Handling currency symbols.
    if is_symbol(token)[0] and not flag:
      sol.append(get_symbol(token, is_symbol(token)[1]))
      flag = True
    
    # Handling percentage symbols.
    if has_percent(token) and not flag:
      sol.append(get_percent(token))
      flag = True

    # Handling all capital tokens (not as abbreviations though).
    if has_allcaps(token) and (len(token) > 1) and not flag:
      sol.append(get_allcaps(token))
      flag = True
    
    # Handling dates.
    if has_date(token)[0] and not flag:
      res = get_date(token, has_date(token)[1])
      if res != 'NONE':
        sol.append(res)
        flag = True

    # Handling 4 digits of numerals; years in general, if there are no symbols, before other numerical strings.
    if has_fournumeric(token) and not flag:
      sol.append(get_fournumeric(token))
      flag = True

    # Handling pure numericals (numbers).
    if has_onlydigits(token.lstrip('0')) and not flag:
      sol.append(get_onlydigits(token))
      flag = True

    # Handling division of numbers.
    if has_division(token) and not flag:
      sol.append(get_division(token))
      flag = True
    
    # Handling time units; for ex: '5:30pm PST' --> 'five thirty p m p s t'
    if has_time(token) and not flag:
      sol.append(get_time(token))
      flag = True

    # Handling extended numericals, for ex: 21st --> 'twenty first'; has to be handled before units.
    if has_extendednums(token) and not flag:
      sol.append(get_extendednums(token))
      flag = True

    # Handling tokens having numeric character with units (has to be done before ISBN; for ex: 823.05 KB).    
    if has_units(token) and not flag:
      sol.append(get_units(token))
      flag = True

    # Handling decimal values.
    if has_decimal(token) and not flag:
      sol.append(get_decimal(token))
      flag = True

    # Handling ISBN Code.
    if has_isbncode(token) and not flag:
      sol.append(get_isbncode(token))
      flag = True

    # If nothing else, then the token should be <self>.
    if not flag:
      sol.append('<self>')

    flag = False    # preparing for the next iteration.

  return sol 


In [94]:
has_decimal('1,565.0')

True

In [95]:
solution(['21112-2302-2121', '3.1', '823.06 KB', '50,000','21st', '1996', '12 July 2008','3:30pm PST', '$40', '17/916', '5.07%', '1,565.0'])

12


['two one one one two sil two three o two sil two one two one',
 'three point one',
 'eight hundred twenty three point o six kilobytes',
 'fifty thousand',
 'twenty first',
 'nineteen ninety six',
 'the twelfth of july two thousand eight',
 'three thirty p m p s t',
 'forty dollars',
 'seventeen nine hundred sixteenths',
 'five point o seven percent',
 'one thousand five hundred sixty five point zero']

In [96]:
has_units('823.06 KB')
get_units('823.06 KB')

'eight hundred twenty three point o six kilobytes'

In [97]:
' '.join('21 '.split())

'21'

In [98]:
def solution_dump(solution_file_path):
  solution_data = []
  for input_sentence in input_data:
    solution_sid = input_sentence['sid']
    print(solution_sid)
    solution_tokens = solution(input_sentence['input_tokens'])
    solution_data.append({'sid':solution_sid,
                          'output_tokens':solution_tokens})

  with open(solution_file_path,'w', encoding='utf-8') as solution_file:
    json.dump(solution_data, solution_file, indent=2, ensure_ascii=False)
    solution_file.close()

In [99]:
solution_dump('solution.json')

0
1
2
14
3
21
08
4
5
6
20
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
10
68
69
70
71
72
73
74
75
76
77
78
22
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
01
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
27
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
14
20
188
189
190
191
192
193
194
195
196
197
198
12
199
200
201
hello
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225


## Error in Solution wrt Ground Truth Data

In [100]:
with open('solution.json','r', encoding='utf-8') as solution_file:
  solution_data = json.load(solution_file)
  solution_file.close()

In [101]:
# Finding count of differences
error_list = {}
count = 0
percentage_error = 0
total_perc_error = 0

for output_sentence in output_data:
    res_sid = output_sentence['sid']
    res_token = output_sentence['output_tokens']

    sol_token  = solution_data[res_sid]['output_tokens']
    for i in range(len(res_token)):
        if (res_token[i] != sol_token[i]):
            count += 1
    percentage_error = 100*count/len(res_token)
    total_perc_error += percentage_error
    error_list[res_sid] = percentage_error
    count = 0

error_list['total'] = total_perc_error/len(error_list)

In [102]:
with open('error_list.json','w') as error_file:
    json.dump(error_list, error_file, indent=2, ensure_ascii=False)
    error_file.close()

# Expected submission

In [103]:
'''
We expect a .py file which takes as input the input file path and the solution file path:

python run_assignment1.py --input_path <path_to_input> --solution_path <path_to_solution>

Once the solution file is generated, we will run a checker script to get the performance metrics:

python run_checker.py --solution_path <path_to_solution> --ground_truth_path <path_to_ground_truth>

'''

'\nWe expect a .py file which takes as input the input file path and the solution file path:\n\npython run_assignment1.py --input_path <path_to_input> --solution_path <path_to_solution>\n\nOnce the solution file is generated, we will run a checker script to get the performance metrics:\n\npython run_checker.py --solution_path <path_to_solution> --ground_truth_path <path_to_ground_truth>\n\n'

# Running on HPC

In [104]:
'''
Students may use the hpc run their code. Please use the following commands to create a conda environment:


module load apps/anaconda/3EnvCreation
conda create --prefix=~/<environment_name> python=3.6
module unload apps/anaconda/3EnvCreation

This is a one-time process. Once the environment is created, the students may wish to install some packages. This can be done as follows:

module load apps/anaconda/3
conda activate ~/<environment_name>
module unload apps/anaconda/3
pip install <package_name> 

It is highly recommended that the students do not install PyTorch in this fashion. The recommended way of configuring PyTorch will be discused in the PyTorch tutorial class scheduled for later this semester.

This assignment will be entirely run on CPU. An interactive CPU job can be run as follows:

qsub -I -P <project_name> -l select=1:ncpus=1:mem=16G -l walltime=<hh:mm:ss>

Once the job is ready, the conda environment would have to be activated again as follows:

module load apps/anaconda/3
conda activate ~/<environment_name>
module unload apps/anaconda/3

Students may be interested to run a jupyter notebook on HPC. Kindly follow the steps from the link below:

https://supercomputing.iitd.ac.in/?softevents=jupyternotebook

'''

'\nStudents may use the hpc run their code. Please use the following commands to create a conda environment:\n\n\nmodule load apps/anaconda/3EnvCreation\nconda create --prefix=~/<environment_name> python=3.6\nmodule unload apps/anaconda/3EnvCreation\n\nThis is a one-time process. Once the environment is created, the students may wish to install some packages. This can be done as follows:\n\nmodule load apps/anaconda/3\nconda activate ~/<environment_name>\nmodule unload apps/anaconda/3\npip install <package_name> \n\nIt is highly recommended that the students do not install PyTorch in this fashion. The recommended way of configuring PyTorch will be discused in the PyTorch tutorial class scheduled for later this semester.\n\nThis assignment will be entirely run on CPU. An interactive CPU job can be run as follows:\n\nqsub -I -P <project_name> -l select=1:ncpus=1:mem=16G -l walltime=<hh:mm:ss>\n\nOnce the job is ready, the conda environment would have to be activated again as follows:\n\n